# 🎓 My Study Assistant

## Welcome to Your Personal AI Learning Companion!

This interactive Jupyter notebook creates a comprehensive AI-powered study assistant that combines Google's Gemini AI with intelligent web search capabilities. The system provides personalized tutoring, automatic image display, motivational support, and session tracking to enhance your learning experience.

Built with Python, this assistant can answer questions across any subject, search for current information, display relevant images, and keep you motivated throughout your study sessions. Simply configure your API keys and start your journey to smarter, more effective studying!

In [ ]:
print("Welcome to Your Personal AI Learning Companion!")

## Package Installation
This cell installs the required Python packages for the notebook including Google's Generative AI SDK, web scraping tools, image processing capabilities, and interactive widgets.
Essential dependencies: `google-generativeai` for AI integration, `requests` and `beautifulsoup4` for web scraping, `pillow` for image handling, and `ipywidgets` for interactive UI components.

In [ ]:
!pip install google-generativeai requests ipywidgets pillow beautifulsoup4 lxml

## Library Imports
This cell imports all necessary libraries for the notebook including Google's Generative AI SDK, web scraping tools (requests, BeautifulSoup), image processing (PIL), and interactive widgets (ipywidgets).
Additional imports include JSON handling, datetime utilities with timezone support, concurrent processing capabilities, and IPython display functions for rich output rendering.

In [ ]:
import google.generativeai as genai
import requests
import json
import random
import time
from datetime import datetime
import pytz
from PIL import Image
from io import BytesIO
import base64
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Javascript
from concurrent.futures import ThreadPoolExecutor

## Timezone Configuration and Utilities
This cell sets up timezone handling for the Indian Standard Time (Asia/Kolkata) and defines utility functions for timestamp formatting.
Functions include `get_local_time()` for current IST time, `format_timestamp()` for HH:MM format, and `format_full_timestamp()` for complete date-time strings.

In [ ]:
LOCAL_TIMEZONE = pytz.timezone('Asia/Kolkata')

def get_local_time():
    """Get current time in local timezone"""
    return datetime.now(LOCAL_TIMEZONE)

def format_timestamp(dt=None):
    """Format timestamp in local timezone"""
    if dt is None:
        dt = get_local_time()
    return dt.strftime("%H:%M")

def format_full_timestamp(dt=None):
    """Format full timestamp with date in local timezone"""
    if dt is None:
        dt = get_local_time()
    return dt.strftime("%Y-%m-%d %H:%M:%S %Z")

## Study Assistant Configuration Class and Setup Interface
This cell defines the `StudyAssistantConfig` class that manages API keys, model initialization, and stores motivational quotes/jokes for the study assistant.
Creates an interactive setup interface with password/text input widgets for Gemini API key, Google Search Engine ID, and Google Search API key configuration.
Includes setup validation, API connection testing, and displays helpful links for obtaining the required API keys from Google AI Studio and Google Custom Search.

In [ ]:
class StudyAssistantConfig:
    def __init__(self):
        self.gemini_api_key = ""
        self.search_engine_id = ""
        self.google_search_api_key = ""
        self.model = None
        self.chat_history = []
        self.motivational_quotes = [
            "🌟 You're doing great! Keep pushing forward!",
            "💪 Every expert was once a beginner. You've got this!",
            "🎯 Focus on progress, not perfection!",
            "🚀 The best time to plant a tree was 20 years ago. The second best time is now!",
            "✨ Your potential is limitless!",
            "🏆 Success is the sum of small efforts repeated daily!",
            "🌈 Believe in yourself and you're halfway there!",
            "💡 Learning is a treasure that follows its owner everywhere!",
            "🔥 You're not just studying, you're investing in your future!",
            "⭐ Champions keep playing until they get it right!"
        ]
        self.study_jokes = [
            "Why don't scientists trust atoms? Because they make up everything! 😄",
            "What do you call a bear with no teeth? A gummy bear! 🐻",
            "Why did the math book look so sad? Because it had too many problems! 📚",
            "What's the best thing about Switzerland? I don't know, but the flag is a big plus! 🇨🇭",
            "Why don't eggs tell jokes? They'd crack each other up! 🥚",
            "What do you call a fake noodle? An impasta! 🍝",
            "Why did the scarecrow win an award? He was outstanding in his field! 🌾",
            "What do you call a sleeping bull? A bulldozer! 🐂",
            "Why don't skeletons fight each other? They don't have the guts! 💀",
            "What's orange and sounds like a parrot? A carrot! 🥕"
        ]

# Initialize configuration
config = StudyAssistantConfig()

# Setup widgets for API configuration
print("🎓 AI Study Assistant & Coach Setup")
print("=" * 50)

# API Key inputs
gemini_key_input = widgets.Password(
    placeholder='Enter your Gemini API Key',
    description='Gemini API:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

search_engine_input = widgets.Text(
    placeholder='Enter your Custom Search Engine ID',
    description='Search Engine ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

google_search_key_input = widgets.Password(
    placeholder='Enter your Google Search API Key',
    description='Google Search API:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

setup_button = widgets.Button(
    description='🚀 Initialize Assistant',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

setup_output = widgets.Output()

def setup_assistant(b):
    with setup_output:
        clear_output()
        try:
            # Configure APIs
            config.gemini_api_key = gemini_key_input.value
            config.search_engine_id = search_engine_input.value
            config.google_search_api_key = google_search_key_input.value

            if not config.gemini_api_key:
                print("❌ Please enter your Gemini API key!")
                return

            if not config.search_engine_id:
                print("❌ Please enter your Search Engine ID!")
                return

            if not config.google_search_api_key:
                print("❌ Please enter your Google Search API key!")
                return

            # Initialize Gemini
            print("🔧 Configuring Gemini API...")
            genai.configure(api_key=config.gemini_api_key)
            config.model = genai.GenerativeModel('gemini-2.0-flash')

            # Test the connection
            print("🧪 Testing API connection...")
            test_response = config.model.generate_content("Hello, say 'API working' if you can see this.")
            print(f"✅ API Test Response: {test_response.text}")

            print("✅ Assistant initialized successfully!")
            print("🎯 Ready to help you study!")
            print("💡 You can now use the chat interface below!")

        except Exception as e:
            print(f"❌ Error initializing assistant: {str(e)}")
            print("Please check your API keys and try again.")

setup_button.on_click(setup_assistant)

# Display setup interface
display(widgets.VBox([
    widgets.HTML("<h3>🔧 Setup Your Study Assistant</h3>"),
    widgets.HTML("<p>Get your API keys from:</p>"),
    widgets.HTML("<ul><li>Gemini API: <a href='https://makersuite.google.com/app/apikey'>Google AI Studio</a></li>"),
    widgets.HTML("<li>Google Search API: <a href='https://developers.google.com/custom-search/v1/overview'>Google Custom Search</a></li></ul>"),
    gemini_key_input,
    search_engine_input,
    google_search_key_input,
    setup_button,
    setup_output
]))

## Study Assistant Core Class Implementation
This cell defines the main `StudyAssistant` class that handles AI interactions, web searching, and chat functionality with methods for prompt engineering, web/image search, and response generation.
Includes intelligent web search triggers, motivational quote/joke dispensers, and chat message formatting with timestamp support for a complete conversational interface.
Implements error handling, debug logging, and creates an assistant instance ready for interactive study sessions with integrated search capabilities.

In [ ]:
class StudyAssistant:
    def __init__(self, config):
        self.config = config
        self.conversation_count = 0

    def get_engineered_prompt(self, user_question, context=""):
        """Create an engineered prompt for better responses"""
        return f"""
        You are an expert AI Study Assistant and Coach. Your role is to help students learn effectively.

        CONTEXT: {context}

        STUDENT QUESTION: {user_question}

        INSTRUCTIONS:
        1. Provide clear, comprehensive explanations
        2. Use examples and analogies when helpful
        3. Break down complex topics into digestible parts
        4. Encourage active learning and critical thinking
        5. If the topic requires visual aids, mention that images will be provided
        6. Be encouraging and supportive
        7. Adapt your explanation to the student's level
        8. Provide practical applications when relevant
        9. Include study tips and memory techniques
        10. End with a question to test understanding

        Respond in a friendly, encouraging tone as a personal tutor would.
        """

    def search_web(self, query, num_results=3):
        """Search the web for relevant information"""
        try:
            print(f"Searching for: {query}")  # Debug line
            url = f"https://www.googleapis.com/customsearch/v1"
            params = {
                'key': self.config.google_search_api_key,
                'cx': self.config.search_engine_id,
                'q': query,
                'num': num_results
            }

            response = requests.get(url, params=params, timeout=10)
            print(f"Search response status: {response.status_code}")  # Debug line

            if response.status_code != 200:
                print(f"Search API error: {response.text}")  # Debug line
                return "Search temporarily unavailable."

            results = response.json()

            search_context = ""
            if 'items' in results:
                for item in results['items']:
                    search_context += f"Title: {item['title']}\n"
                    search_context += f"Snippet: {item['snippet']}\n\n"
                print(f"Found {len(results['items'])} search results")  # Debug line
            else:
                print("No search results found")  # Debug line

            return search_context
        except Exception as e:
            print(f"Search error: {str(e)}")  # Debug line
            return "Search temporarily unavailable."

    def search_images(self, query, num_images=3):
        """Search for relevant images"""
        try:
            url = f"https://www.googleapis.com/customsearch/v1"
            params = {
                'key': self.config.google_search_api_key,
                'cx': self.config.search_engine_id,
                'q': query,
                'searchType': 'image',
                'num': num_images,
                'safe': 'active'
            }

            response = requests.get(url, params=params)
            results = response.json()

            images = []
            if 'items' in results:
                for item in results['items']:
                    images.append({
                        'url': item['link'],
                        'title': item['title'],
                        'context': item.get('snippet', '')
                    })

            return images
        except Exception as e:
            return []

    def get_motivation(self):
        """Get a random motivational quote"""
        return random.choice(self.config.motivational_quotes)

    def get_joke(self):
        """Get a random study-related joke"""
        return random.choice(self.config.study_jokes)

    def should_search_web(self, question):
        """Determine if web search is needed"""
        search_triggers = [
            'current', 'recent', 'latest', 'news', 'today', 'this year',
            'what happened', 'who is', 'where is', 'when did', 'statistics',
            'data', 'research', 'study shows', 'according to', 'real-time'
        ]

        question_lower = question.lower()
        return any(trigger in question_lower for trigger in search_triggers)

    def generate_response(self, user_input):
        """Generate AI response with web search if needed"""
        try:
            print(f"Processing question: {user_input}")  # Debug line

            # Check if web search is needed
            search_context = ""
            if self.should_search_web(user_input):
                print("Searching web...")  # Debug line
                search_context = self.search_web(user_input)
                print(f"Search context: {search_context[:100]}...")  # Debug line

            # Generate response using Gemini
            prompt = self.get_engineered_prompt(user_input, search_context)
            print("Generating AI response...")  # Debug line

            response = self.config.model.generate_content(prompt)
            print("Response generated successfully!")  # Debug line

            return response.text

        except Exception as e:
            print(f"Error in generate_response: {str(e)}")  # Debug line
            return f"Sorry, I encountered an error: {str(e)}. Please check your API keys and try again."

    def format_message(self, sender, message, timestamp=None):
        """Format message for chat display"""
        if timestamp is None:
            timestamp = datetime.now().strftime("%H:%M")

        if sender == "user":
            return f"""
            <div style="margin: 10px 0; text-align: right;">
                <div style="display: inline-block; background: #007bff; color: white; padding: 10px 15px; border-radius: 18px; max-width: 70%; text-align: left;">
                    {message}
                </div>
                <div style="font-size: 0.8em; color: #666; margin-top: 5px;">You • {timestamp}</div>
            </div>
            """
        else:
            return f"""
            <div style="margin: 10px 0; text-align: left;">
                <div style="display: inline-block; background: #f1f1f1; color: #333; padding: 10px 15px; border-radius: 18px; max-width: 70%; text-align: left;">
                    {message}
                </div>
                <div style="font-size: 0.8em; color: #666; margin-top: 5px;">AI Assistant • {timestamp}</div>
            </div>
            """

    def display_images(self, images):
        """Display images in chat"""
        if not images:
            return ""

        html = "<div style='margin: 10px 0;'>"
        for img in images[:3]:  # Limit to 3 images
            html += f"""
            <div style="display: inline-block; margin: 5px; text-align: center;">
                <img src="{img['url']}" style="max-width: 150px; max-height: 150px; border-radius: 8px;" onerror="this.style.display='none'"/>
                <div style="font-size: 0.8em; color: #666; max-width: 150px; word-wrap: break-word;">{img['title'][:50]}...</div>
            </div>
            """
        html += "</div>"
        return html

# Initialize the assistant
assistant = StudyAssistant(config)

## Study Session Tracker
This cell implements a `StudyTracker` class to monitor study sessions with start/end times, question counts, and topic tracking capabilities.
Creates interactive buttons for starting/ending study sessions with real-time feedback and session statistics display.

In [ ]:
class StudyTracker:
    def __init__(self):
        self.session_start = None
        self.topics_studied = []
        self.questions_asked = 0

    def start_session(self):
        self.session_start = datetime.now()
        print(f"📚 Study session started at {self.session_start.strftime('%H:%M')}")

    def end_session(self):
        if self.session_start:
            duration = datetime.now() - self.session_start
            print(f"🎯 Study session ended!")
            print(f"Duration: {duration}")
            print(f"Questions asked: {self.questions_asked}")
            print(f"Topics studied: {', '.join(self.topics_studied) if self.topics_studied else 'None'}")
        else:
            print("No active study session!")

    def add_topic(self, topic):
        if topic not in self.topics_studied:
            self.topics_studied.append(topic)

    def increment_questions(self):
        self.questions_asked += 1

# Initialize tracker
tracker = StudyTracker()

# Study session controls
session_start_btn = widgets.Button(description='📚 Start Session', button_style='success')
session_end_btn = widgets.Button(description='🏁 End Session', button_style='danger')
session_output = widgets.Output()

def start_study_session(b):
    with session_output:
        clear_output()
        tracker.start_session()

def end_study_session(b):
    with session_output:
        clear_output()
        tracker.end_session()

session_start_btn.on_click(start_study_session)
session_end_btn.on_click(end_study_session)

display(widgets.VBox([
    widgets.HTML("<h3>📊 Study Session Tracker</h3>"),
    widgets.HBox([session_start_btn, session_end_btn]),
    session_output
]))

## Interactive Chat Interface Implementation
This cell creates the complete chat UI with input field, send button, and special buttons for motivation/jokes, along with a scrollable chat output area.
Handles message sending, typing indicators, chat history management, and automatic image display with proper error handling and user feedback.
Includes JavaScript for auto-scrolling, Enter key support, and provides usage tips with styling for a professional chat experience.

In [ ]:
from ast import Add
chat_output = widgets.Output(layout=widgets.Layout(height='400px', overflow='auto'))
user_input = widgets.Text(
    placeholder='Ask me anything about your studies...',
    layout=widgets.Layout(width='70%')
)
send_button = widgets.Button(
    description='Send',
    button_style='primary',
    layout=widgets.Layout(width='10%')
)
motivate_button = widgets.Button(
    description='💪 Motivate',
    button_style='info',
    layout=widgets.Layout(width='10%')
)
joke_button = widgets.Button(
    description='😄 Joke',
    button_style='warning',
    layout=widgets.Layout(width='10%')
)

def scroll_to_bottom():
    """Scroll chat to bottom"""
    display(Javascript("""
    setTimeout(function() {
        var output = document.querySelector('.jp-OutputArea-output');
        if (output) {
            output.scrollTop = output.scrollHeight;
        }
    }, 100);
    """))

def send_message(b=None):
    """Send user message and get AI response"""
    if not config.model:
        with chat_output:
            print("❌ Please initialize the assistant first!")
        return


    message = user_input.value.strip()
    tracker.add_topic(message)
    if not message:
        return


    # Clear input
    user_input.value = ""
    tracker.increment_questions()

    # Display user message
    timestamp = datetime.now().strftime("%H:%M")
    with chat_output:
        display(HTML(assistant.format_message("user", message, timestamp)))
        config.chat_history.append(assistant.format_message("user", message, timestamp))
        scroll_to_bottom()


    # Show typing indicator
    with chat_output:
        typing_html = """
        <div style="margin: 10px 0; text-align: left;">
            <div style="display: inline-block; background: #f1f1f1; color: #333; padding: 10px 15px; border-radius: 18px;">
                <span style="animation: blink 1s infinite;">🤖 AI is thinking...</span>
            </div>
        </div>
        <style>
        @keyframes blink {
            0%, 50% { opacity: 1; }
            51%, 100% { opacity: 0.5; }
        }
        </style>
        """
        display(HTML(typing_html))
        scroll_to_bottom()

    # Generate response directly (not in thread for better compatibility)
    try:
        # Generate AI response
        ai_response = assistant.generate_response(message)

        # Search for relevant images
        images = assistant.search_images(message)

        # Clear typing indicator and display response
        with chat_output:
            # Remove typing indicator by clearing last output
            clear_output(wait=True)

            # Redisplay all chat history
            for msg in config.chat_history:
                display(HTML(msg))

            # Display AI response
            display(HTML(assistant.format_message("assistant", ai_response)))
            config.chat_history.append(assistant.format_message("assistant", ai_response))

            # Display images if available
            if images:
                image_html = assistant.display_images(images)
                display(HTML(image_html))
                config.chat_history.append(image_html)

        # Update question counter
        assistant.conversation_count += 1

        # Scroll to bottom
        scroll_to_bottom()

    except Exception as e:
        with chat_output:
            clear_output(wait=True)
            # Redisplay chat history
            for msg in config.chat_history:
                display(HTML(msg))
            error_msg = f"Sorry, I encountered an error: {str(e)}"
            display(HTML(assistant.format_message("assistant", error_msg)))
            config.chat_history.append(assistant.format_message("assistant", error_msg))

def send_motivation(b):
    """Send motivational message"""
    if not config.model:
        with chat_output:
            print("❌ Please initialize the assistant first!")
        return

    motivation = assistant.get_motivation()
    with chat_output:
        timestamp = datetime.now().strftime("%H:%M")
        display(HTML(assistant.format_message("assistant", motivation, timestamp)))
        config.chat_history.append(assistant.format_message("assistant", motivation, timestamp))

    scroll_to_bottom()

def send_joke(b):
    """Send a joke"""
    if not config.model:
        with chat_output:
            print("❌ Please initialize the assistant first!")
        return

    joke = assistant.get_joke()
    with chat_output:
        timestamp = datetime.now().strftime("%H:%M")
        display(HTML(assistant.format_message("assistant", joke, timestamp)))
        config.chat_history.append(assistant.format_message("assistant", joke, timestamp))

    scroll_to_bottom()

# Connect button events
send_button.on_click(send_message)
motivate_button.on_click(send_motivation)
joke_button.on_click(send_joke)

# Handle Enter key
def handle_enter(sender):
    send_message()

user_input.on_submit(handle_enter)

# Display chat interface
display(widgets.VBox([
    widgets.HTML("<h3>💬 Chat with Your Study Assistant</h3>"),
    chat_output,
    widgets.HBox([user_input, send_button, motivate_button, joke_button]),
    widgets.HTML("""
    <div style="margin-top: 10px; padding: 10px; background: #2905f7; border-radius: 8px;">
        <strong>💡 Tips:</strong><br>
        • Ask specific questions about any topic<br>
        • Request explanations, examples, or study tips<br>
        • Use the Motivate button when you need encouragement<br>
        • Use the Joke button to lighten the mood<br>
        • The assistant will search the web for current information when needed<br>
        • Images related to your questions will be displayed automatically
    </div>
    """),
    widgets.HTML("""
    <style>
    .jp-OutputArea-output {
        max-height: 400px;
        overflow-y: auto;
        scroll-behavior: smooth;
    }
    </style>
    """)
]))

## Conclusion
This AI Study Assistant notebook provides a complete learning companion with intelligent tutoring, web search integration, and motivational features to enhance your educational journey.
With session tracking and personalized responses, you now have a powerful tool to make studying more effective, engaging, and enjoyable - happy learning!